In [1]:
import numpy as np
import pandas as pd

# Re-create the KNN Algorithm
## Steven Glover
***
To recreate the KNN algorithm I created a series of functions that were leveraged by a single wrapper function. In this notebook, I first used Iris data sets to test this functionality. I prototyped the titanic dataset in another notebook, but the code included below and in the attached script are identical. I demonstrate the wrapper function using both the Titanic dataset and the Iris dataset at the end of this workbook. <br>
<br>**Preprocessing:**
1.	data_type_preprocessing: The function breaks the data frame into a train / split, converts the categorical variables to dummies, and normalizes the X-test and X-Train subsets to be on a zero to one scale.  To manage different data types, the function will behave differently depending on the input parameters. The var_type parameter indicates whether the inputted data frame consists of only categorical variables, only continuous variables, or a mixture between the two. If the function is given only categorical variables, the categorical variables will be converted to dummies and the test / train split along with the column names is returned. If the function is given only continuous variables, the variables will be normalized, and the test / train split along with the column names is returned. If the function is given a mixed data frame the user must also include lists identifying which columns are continuous and categorical in the continuous_list and categorical_list arguments. For mixed data frames, the function will return the test / train, the column names, the indexes of the continuous variables and the indexes of the categorical variables.  <br>
<br>**Distance Measures:**
2.	minkowski_distance – will compute the minkowki distance between two arrays
3.	minkowski_point_array : (To be used in conjunction with the Gower Distance)  - The function returns an array of the Minkowski distances between each continuous point of two arrays. This is needed because the Gower Distance requires that these point distance measures are normalized before the Gower Distance is calculated. 
4.	jaccard_distance – will compute the jaccard distance for categorical variables
5.	gower_distance - The function returns the distances needed to calculate the Gower distance.  Note, the return is not a single distance measure because of an additional normalizing step for the continuous distance measures.  The function returns 3 variables: 1 the Jaccard distance measure for the categorical variables, 2.  an array of Minkowski distance measure between each continuous point, 3. the denominator for the Gower distance calculation. <br>
<br>**KNN Predictions:**
6.	knn_predictions: The function calculates the distance measures for each observation in the test set to each observation in the training set. An array of distance measures is created for each observation in the test. The function then returns the indexes of the K smallest distances in the array. Using these indexes, we are able to determine the y values associated with the smallest distances measures, which are stored in another array. We then obtain the vote for the prediction using a combination of the numpy argmax function and the numpy unique function. This process is repeated for each of the observation in the test set and the predictions are stored to a list with every iteration. <br>
<br>**Prediction Accuracy Report**
7.	The function returns the accuracy score, precision / recall tables, and a confusion matrix for the predictions. <br>
<br>**The Wrapper Function**
8.	Complete_KNN: The function wraps all the above functions together to provide preprocessing, generate predictions using the KNN algorithm, and an accuracy report. The function takes the following arguments:
    - Df – the data frame the user is working with
    - y – the name of the y variable as a string
    - var_type – identifies the datatypes of the predictors and takes the following inputs: continuous, categorical or mixed. It is continuous by default.
    - continuous_list = This field is required for data frames of mixed datatypes, such as the titanic dataset. The argument takes a list with the column names of the continuous variables.
    - categorical_list = This field is required for data frames of mixed datatypes. The argument takes a list with the column names of the categorical variables.
    - k = the number of nearest neighbors to calculate
     - distance_measure = is the distance measure used to determine the nearest neighbors. It is ‘euclidean’ by default. It will take ‘euclidean’, ‘manhattan’, ‘jaccard’, and ‘gower.’  

**<font color = red> Please note: </font> ** the user is required to know which measures to use for which data type. Errors may result if the user tries to compute the Gower distance on an all continuous or all categorical data frame. Additionally, using the Jaccard distance for an all continuous data frame will produce errors as well. 


## 1) Import and Preprocess the Data

In [9]:
from sklearn import datasets
#import the Iris Dataset
iris = datasets.load_iris()
#create a dataframe of the iris dataset
data = pd.DataFrame(data=iris.data, columns=['SEPAL_LENGTH','SEPAL_WIDTH','PETAL_LENGTH','PETAL_WIDTH'])
data['TARGET'] = iris.target

#print Information about the dataset
print('columns: ',data.columns,'\n')
print('shape: ', data.shape,'\n')
print('null: \n', pd.isnull(data).sum())
print('-------------------------------')
print('Unique Values: ', data.nunique())

for i in data.columns.tolist():
    print(i,'\n',data[i].unique())
    print('---------------')
    
display(data.head())

columns:  Index(['SEPAL_LENGTH', 'SEPAL_WIDTH', 'PETAL_LENGTH', 'PETAL_WIDTH', 'TARGET'], dtype='object') 

shape:  (150, 5) 

null: 
 SEPAL_LENGTH    0
SEPAL_WIDTH     0
PETAL_LENGTH    0
PETAL_WIDTH     0
TARGET          0
dtype: int64
-------------------------------
Unique Values:  SEPAL_LENGTH    35
SEPAL_WIDTH     23
PETAL_LENGTH    43
PETAL_WIDTH     22
TARGET           3
dtype: int64
SEPAL_LENGTH 
 [ 5.1  4.9  4.7  4.6  5.   5.4  4.4  4.8  4.3  5.8  5.7  5.2  5.5  4.5  5.3
  7.   6.4  6.9  6.5  6.3  6.6  5.9  6.   6.1  5.6  6.7  6.2  6.8  7.1  7.6
  7.3  7.2  7.7  7.4  7.9]
---------------
SEPAL_WIDTH 
 [ 3.5  3.   3.2  3.1  3.6  3.9  3.4  2.9  3.7  4.   4.4  3.8  3.3  4.1  4.2
  2.3  2.8  2.4  2.7  2.   2.2  2.5  2.6]
---------------
PETAL_LENGTH 
 [ 1.4  1.3  1.5  1.7  1.6  1.1  1.2  1.   1.9  4.7  4.5  4.9  4.   4.6  3.3
  3.9  3.5  4.2  3.6  4.4  4.1  4.8  4.3  5.   3.8  3.7  5.1  3.   6.   5.9
  5.6  5.8  6.6  6.3  6.1  5.3  5.5  6.7  6.9  5.7  6.4  5.4  5.2]
--------------

,SEPAL_LENGTH,SEPAL_WIDTH,PETAL_LENGTH,PETAL_WIDTH,TARGET
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## <font color = blue> The following functions and code is the prototyping for the Complete_KNN function in the .py script file </font>
The KNN script file will be called at the end of the file to ensure that it works. 

# Preprocessing & Train Test Split
***

In [10]:
def data_type_preprocessing(df,y, var_type = 'continuous',
                            continuous_list = None, categorical_list = None, categorical_only= False):
    
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.preprocessing import StandardScaler
    from sklearn.cross_validation import train_test_split
    ''' The following function will preprocess the data and create a train test split.
    It will be able to handle dataframes that are all categorical, all continuous, or mixed.
    it will create dummies for categorical variables. It will normalize continous variables.
    it will also create a train test split. For mixed dataframes it will return the indexes of 
    the continuous and categorical variables.
    
    var_type
    --------
    var_type = 'continuous' (by default. Indicates only continous variables in the dataframe)
    var_type = 'categorical' ('Indicates only a categorical dataframe)
    var_type = 'Mixed ('Indicates both categorical and continous variables present. 
                        Requires both categorical list, and continuous lists.)
    '''
    
    #get variable names from the function
    #------------------------------------
    
    #get the y col
    y_col = y
    
    #create the y array 
    y = np.array(df[y].astype(str))
    
    """ Determine the type variables we are working with and 
    process accrodingly"""
    
    if var_type == 'mixed':
        #get categorical & continuous variables
        cont = continuous_list
        cat = categorical_list
        
        #subset continuous
        cont_df = df[cont]
        
        #get the length of continuous and categorical to slice arrays
        split_position = len(cont)
        
        #get dummies for categorical variables
        cat_df = pd.get_dummies(df[cat].astype(str))
    
        #recreate the dataframe with dummies
        X = pd.merge(cont_df,cat_df, how = 'left', left_index=True, right_index = True)
        
    elif var_type == 'categorical':
        #create a dataframe of dummy variables that do not include y
        X = pd.get_dummies(df[categorical_list])
        
    elif var_type == 'continuous':
        X = df[continuous_list]
    
    
    # Get column names
    col_names = X.columns
    X = np.array(X)
    
    # Create the train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    
    #preprocess continuous variables
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
        
    if var_type == 'mixed':
        #idenify the array index of categorical and continuous columns
        cont_idx = [i for i in range(split_position)]
        cat_idx = [i for i in range(split_position, col_names.shape[0])] 
        return X_train, X_test, y_train, y_test, col_names, cont_idx, cat_idx
    else:
        return X_train, X_test, y_train, y_test, col_names

In [11]:
X_train, X_test, y_train, y_test, col_names = data_type_preprocessing(data, y = 'TARGET')

In [12]:
pd.DataFrame(data = X_train,columns=col_names)

,SEPAL_LENGTH,SEPAL_WIDTH,PETAL_LENGTH,PETAL_WIDTH
0,0.722222,0.458333,0.745763,0.833333
1,0.250000,0.875000,0.084746,0.000000
2,0.583333,0.291667,0.728814,0.750000
3,0.166667,0.458333,0.084746,0.000000
4,0.472222,0.375000,0.593220,0.583333
5,0.805556,0.416667,0.813559,0.625000
6,0.250000,0.291667,0.491525,0.541667
7,0.388889,1.000000,0.084746,0.125000
8,0.194444,0.583333,0.101695,0.125000
9,0.305556,0.708333,0.084746,0.041667



# Distance Measures 
***

### Minkowski

In [13]:
from scipy.spatial.distance import minkowski
from scipy.spatial.distance import jaccard

### Minkowski Distance

In [14]:
A = X_train[0,:]
B = X_train[1,:]

In [15]:
# True minkowski Distance between two arrays
def minkowski_distance(A, B, p =2):
    p = 2
    diff_abs = np.abs(A - B)
    to_the_p = diff_abs**p
    sum_dist = sum(to_the_p)
    sum_div_p = sum_dist**(1/p)
    return sum_div_p

minkowski_distance(A, B, p =2)

1.2361200547612616

In [16]:
#scipy check
minkowski(A , B, p= 2)

1.2361200547612616

In [17]:
# Minkowski Distance between two points for the Gower
def minkowski_point_array(A, B, p = 2):
    '''This function will be used in the gower distance calculation.
    it differs from the true minkowsi in that calculates the minkowski distance
    between two points rather than between two arrays'''
    #absolute value of the difference
    diff_abs = np.abs(A - B)
    #to the p
    to_the_p = diff_abs**p
    #each point in the array raised to the value of 1/p
    dist_raised_1div_p = to_the_p**(1/p)
    return dist_raised_1div_p

In [18]:
minkowski_point_array(A, B, p = 2)

array([ 0.47222222,  0.41666667,  0.66101695,  0.83333333])

### Jaccard Distance

In [19]:
def jaccard_distance(A, B):
    a = []
    b = []
    c = []
    for index in range(len(A)):
        if A[index] == 1 and B[index] == 1:
            a.append(1)
        elif A[index] == 1 and B[index] == 0:
            b.append(1)
        elif A[index] == 0 and B[index] == 1:
            c.append(1)
    return (sum(b) + sum(c))/(sum(a) + sum(b) + sum(c))

### Gower Distance function
***

In [20]:
def gower_distance(A, B, cont_idx, cat_idx, p = 2):
    """This function will return the jaccard distance calculation, an array of the minkowski point calculations.
    note: the minkowski 'point' distances are not summed. Just an array of the gower distances from point to point
    is returned. The function was built this way because we need to nomalize the gower array before computing the
    gower distances. The function also returns the demominator for the gower distance calculations
    (+ 1 for each continous and a 1 if categorical variables exist)"""
    denominator = len(cont_idx) + 1
    return jaccard_distance(A[cat_idx], B[cat_idx]), minkowski_point_array(A[cont_idx], B[cont_idx],p = 2), denominator

# The  KNN Predictions
***

In [21]:
# 1. calcualte the distances and store them to a list
def knn_predictions(X_train, X_test, y_train, y_test, col_names, 
                    cont_idx = None, cat_idx = None, k = 5, distance = 'gower'):
    
    """
    Distance_measures
    -----------------
    1. 'gower'
    2. 'jaccard'
    3. 'euclidean'
    4. 'manhattan'
    """
    
    from sklearn.preprocessing import MinMaxScaler
    predictions = []
    for test_index in range(X_test.shape[0]):
        test = X_test[test_index,:]
        distances = []
        
        """The following section will populate the distance for each observation in the training
        set to the current test observation. Due to the normalization of the distances prior to 
        concatenation with the jaccard caculation, the gower distance has a much lengthy procedure."""
        
        if distance == 'gower':
        #calculate the distance of the test line to all of the other training lines
            for train_index in range(X_train.shape[0]):
                #indentify the training line
                train = X_train[train_index,:]

                #Calcuates the Gower Distance

                '''get the an the jaccard distances calculations, 
                an array of the minkowski distance calculations for each point, and 
                the denominator'''

                jaccard, gower_point_dist, denominator = gower_distance(test, train, cont_idx, cat_idx)
                #store each calculation to an array
                if train_index == 0:
                    jaccard_array = np.array(jaccard).reshape(1,1)
                    gower_array = gower_point_dist.reshape(1,len(cont_idx))
                else:
                    jaccard_array = np.append(jaccard_array, np.array(jaccard).reshape(1,1),axis=0)
                    gower_array = np.append(gower_array,gower_point_dist.reshape(1,len(cont_idx)),axis=0)

            #normalize the gower distance to be on a zero to one scale
            scaler = MinMaxScaler()
            gower_array = scaler.fit_transform(gower_array)

            #combine the jackard distance calcuation wtih the normalized gower array
            normalized_gower_array = np.hstack((jaccard_array, gower_array))
            '''CALCULATE ALL THE GOWER DISTANCES LINE BY LINE AND APPEND TO A LIST''' 
            for i in range(normalized_gower_array.shape[0]):
                distances.append((sum(normalized_gower_array[i,:]))/denominator)
                
        #create an array of manhattan distances        
        elif distance == 'manhattan':
            for train_index in range(X_train.shape[0]):
                train = X_train[train_index,:]
                distances.append(minkowski_distance(test, train, p =1))
                    
        #create an array of euclidean distances 
        elif distance == 'euclidean':
            for train_index in range(X_train.shape[0]):
                train = X_train[train_index,:]
                distances.append(minkowski_distance(test, train, p =2))
                    
        #create an array of manhattan distances 
        elif distance == 'jaccard':
            for train_index in range(X_train.shape[0]):
                train = X_train[train_index,:]
                distances.append(jaccard_distance(test, train))
                    
        # 3. get the indexes for the nearest neighbors
        prediction_index = np.array(distances).argsort()[:k]

        # 4. get all the predictions values
        prediction_values = y_train[prediction_index]

        # 5. Use the numpy unique function to determine which prediction appeared the most
        # returns the position in the unique array with the maximum count
        max_index = np.argmax(np.unique(prediction_values , return_counts=True)[1])

        # 6. returns the value of the maximum count
        prediction = np.unique(prediction_values , return_counts=True)[0][max_index]

        # 7. store the prediction to the list
        predictions.append(prediction)
        
    return np.array(predictions) 

In [22]:
predictions = knn_predictions(X_train, X_test, y_train, y_test, col_names, k = 5, distance = 'euclidean')

# Classification Report
***

In [23]:
def accuaracy_report(y_test, predictions):
    from sklearn.metrics import classification_report, confusion_matrix
    print('The Accuracy Score is: ',np.array(np.sum(np.equal(predictions, y_test))) / y_test.shape[0],'\n')
    print('The Classification Report')
    print('-------------------------')
    print(classification_report(y_test, predictions),'\n')
    print('The Confusion Matrix')
    print('-------------------------')
    print(pd.DataFrame(confusion_matrix(y_test, predictions)).apply(lambda x: x / sum(x), axis=1))

In [24]:
accuaracy_report(y_test, predictions)

The Accuracy Score is:  0.966666666667 

The Classification Report
-------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        15
          1       1.00      0.90      0.95        10
          2       0.83      1.00      0.91         5

avg / total       0.97      0.97      0.97        30
 

The Confusion Matrix
-------------------------
     0    1    2
0  1.0  0.0  0.0
1  0.0  0.9  0.1
2  0.0  0.0  1.0


# Bringing it all together for a complete KNN Function
The below was used to draft the wrapper function for the KNN.
I added all of these functions to a single script to be used as a library
***

In [25]:
"""The Complete KNN Wrapper Fuction"""
"""--------------------------------"""
def Complete_KNN(df, y, var_type = 'continuous',
                 continuous_list = None, categorical_list = None, categorical_only= False,
                 k = 5, distance_measure = 'euclidean'):
    
    #import packages
    import numpy as np
    import pandas as pd
    
    # the index variables will be populated if we are using a mixed type
    cont_idx = None
    cat_idx = None
    
    # Step 1 - Preprocessing
    if var_type == 'mixed':
        X_train, X_test, y_train, y_test, col_names, cont_idx, cat_idx = data_type_preprocessing(df,y, var_type = var_type,
                                                                                        continuous_list = continuous_list,
                                                                                        categorical_list = categorical_list,
                                                                                        categorical_only= categorical_only)
    else:
        X_train, X_test, y_train, y_test, col_names = data_type_preprocessing(df,y, var_type = var_type,
                                                                                        continuous_list = continuous_list,
                                                                                        categorical_list = categorical_list,
                                                                                        categorical_only= categorical_only)
        
    # Step 2 - Generate Predictions
    predictions = knn_predictions(X_train, X_test, y_train, y_test, col_names, 
                                 cont_idx = cont_idx, cat_idx = cat_idx,
                                 k = k, distance = distance_measure)
    
    
    #Step 3 - Print Accuarcy Report
    accuaracy_report(y_test, predictions)
    
    # Step 4 - Return Predictions
    return predictions

### Using the Complete_KNN function from the KNN_Glover.py Script File

In [26]:
from KNN_Glover import Complete_KNN
Complete_KNN(data, y = 'TARGET', var_type = 'continuous',
                 continuous_list = None, categorical_list = None, categorical_only= False,
                 k = 5, distance_measure = 'euclidean' )

The Accuracy Score is:  0.966666666667 

The Classification Report
-------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         7
          1       1.00      0.92      0.96        13
          2       0.91      1.00      0.95        10

avg / total       0.97      0.97      0.97        30
 

The Confusion Matrix
-------------------------
     0         1         2
0  1.0  0.000000  0.000000
1  0.0  0.923077  0.076923
2  0.0  0.000000  1.000000


array(['1', '2', '0', '1', '0', '1', '2', '2', '2', '2', '2', '0', '2',
       '1', '1', '0', '1', '1', '0', '1', '2', '2', '1', '1', '1', '0',
       '1', '2', '0', '2'],
      dtype='<U1')

<br>
<br>
<br>
<br>
***
# Titanic Dataset
***

## 1) Import and Preprocess the Data

In [27]:
data = pd.read_csv('C:/Users/Steven Glover/Jupyter Notebooks/Fall Semester/KNN/titanic3.csv')
print('columns: ',data.columns,'\n')
print('shape: ', data.shape,'\n')
print('null: \n', pd.isnull(data).sum())
print('-------------------------------')
print('Unique Values: ', data.nunique())

columns:  Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object') 

shape:  (1310, 14) 

null: 
 pclass          1
survived        1
name            1
sex             1
age           264
sibsp           1
parch           1
ticket          1
fare            2
cabin        1015
embarked        3
boat          824
body         1189
home.dest     565
dtype: int64
-------------------------------
Unique Values:  pclass          3
survived        2
name         1307
sex             2
age            98
sibsp           7
parch           8
ticket        929
fare          281
cabin         186
embarked        3
boat           27
body          121
home.dest     369
dtype: int64


### Drop Name, Cabin, Ticket, Boat,  Get Rid of Null Values, and Inspect Unique Values

In [28]:
data = data.drop(['name','cabin','ticket','boat','home.dest','body'], axis = 1)
data = data.dropna()
data.info()

for i in data.columns.tolist():
    print(i,'\n',data[i].unique())
    print('---------------')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1308
Data columns (total 8 columns):
pclass      1043 non-null float64
survived    1043 non-null float64
sex         1043 non-null object
age         1043 non-null float64
sibsp       1043 non-null float64
parch       1043 non-null float64
fare        1043 non-null float64
embarked    1043 non-null object
dtypes: float64(6), object(2)
memory usage: 73.3+ KB
pclass 
 [ 1.  2.  3.]
---------------
survived 
 [ 1.  0.]
---------------
sex 
 ['female' 'male']
---------------
age 
 [ 29.       0.9167   2.      30.      25.      48.      63.      39.      53.
  71.      47.      18.      24.      26.      80.      50.      32.      36.
  37.      42.      19.      35.      28.      45.      40.      58.      22.
  41.      44.      59.      60.      33.      17.      11.      14.      49.
  76.      46.      27.      64.      55.      70.      38.      51.      31.
   4.      54.      23.      43.      52.      16.      32.

# Use the Complete_KNN Algorithm via KNN.py Script

In [29]:
from KNN import Complete_KNN

Complete_KNN(data, y = 'survived', var_type = 'mixed',
                 continuous_list = ['age', 'sibsp', 'parch', 'fare'],
                 categorical_list = ['pclass','sex','embarked'],
                 categorical_only= False,
                 k = 5, distance_measure = 'gower' )

The Accuracy Score is:  0.803827751196 

The Classification Report
-------------------------
             precision    recall  f1-score   support

        0.0       0.83      0.85      0.84       123
        1.0       0.77      0.74      0.76        86

avg / total       0.80      0.80      0.80       209
 

The Confusion Matrix
-------------------------
          0         1
0  0.845528  0.154472
1  0.255814  0.744186


array(['0.0', '0.0', '0.0', '0.0', '1.0', '1.0', '1.0', '1.0', '0.0',
       '0.0', '1.0', '1.0', '1.0', '1.0', '1.0', '0.0', '0.0', '0.0',
       '1.0', '0.0', '1.0', '0.0', '0.0', '0.0', '0.0', '1.0', '1.0',
       '0.0', '1.0', '1.0', '0.0', '1.0', '0.0', '1.0', '0.0', '1.0',
       '0.0', '1.0', '0.0', '1.0', '1.0', '0.0', '1.0', '1.0', '0.0',
       '0.0', '1.0', '0.0', '0.0', '0.0', '0.0', '1.0', '1.0', '1.0',
       '1.0', '0.0', '0.0', '0.0', '1.0', '0.0', '0.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '0.0', '1.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '0.0', '0.0',
       '0.0', '1.0', '0.0', '1.0', '1.0', '0.0', '1.0', '1.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '0.0', '0.0',
       '1.0', '0.0', '1.0', '0.0', '1.0', '1.0', '0.0', '0.0', '1.0',
       '0.0', '0.0', '0.0', '1.0', '1.0', '0.0', '1.0', '0.0', '0.0',
       '0.0', '1.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '1.0',